Assignment

Data Science Masters

Q1. You are work#ng on a mach#ne learn#ng project where you have a dataset conta#n#ng numer#cal and
categor#cal features. You have #dent#f#ed that some of the features are h#ghly correlated and there are
m#ss#ng values #n some of the columns. You want to bu#ld a p#pel#ne that automates the feature
eng#neer#ng process and handles the m#ss#ng valuesD
Des#gn a p#pel#ne that #ncludes the follow#ng steps"
Use an automated feature select#on method to #dent#fy the #mportant features #n the datasetC
Create a numer#cal p#pel#ne that #ncludes the follow#ng steps"
Impute the m#ss#ng values #n the numer#cal columns us#ng the mean of the column valuesC
Scale the numer#cal columns us#ng standard#sat#onC
Create a categor#cal p#pel#ne that #ncludes the follow#ng steps"
Impute the m#ss#ng values #n the categor#cal columns us#ng the most frequent value of the columnC
One-hot encode the categor#cal columnsC
Comb#ne the numer#cal and categor#cal p#pel#nes us#ng a ColumnTransformerC
Use a Random Forest Class#f#er to bu#ld the f#nal modelC
Evaluate the accuracy of the model on the test datasetD
Note! Your solut#on should #nclude code sn#ppets for each step of the p#pel#ne, and a br#ef explanat#on of
each step. You should also prov#de an #nterpretat#on of the results and suggest poss#ble #mprovements for
the p#pel#neD
Q2. Bu#ld a p#pel#ne that #ncludes a random forest class#f#er and a log#st#c regress#on class#f#er, and then
use a vot#ng class#f#er to comb#ne the#r pred#ct#ons. Tra#n the p#pel#ne on the #r#s dataset and evaluate #ts
accuracy.

In [69]:
import pandas as pd
import numpy as np
from seaborn import load_dataset
from sklearn.impute import SimpleImputer ## HAndling Missing Values
from sklearn.preprocessing import StandardScaler # HAndling Feature Scaling
from sklearn.preprocessing import OrdinalEncoder # Ordinal Encoding
## pipelines
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
import pandas as pd
import numpy as np
from seaborn import load_dataset
from sklearn.preprocessing import StandardScaler,LabelEncoder,OneHotEncoder
from sklearn.feature_selection import VarianceThreshold
from sklearn.feature_selection import mutual_info_classif
from sklearn.impute import SimpleImputer
from sklearn.compose import ColumnTransformer
from sklearn.ensemble import RandomForestClassifier,VotingClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.pipeline import Pipeline, make_pipeline
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score,confusion_matrix,classification_report
from warnings import filterwarnings as fw
fw('ignore')

In [31]:
df=load_dataset('planets')

In [32]:
df.head()

,method,number,orbital_period,mass,distance,year
0,Radial Velocity,1,269.300,7.10,77.40,2006
1,Radial Velocity,1,874.774,2.21,56.95,2008
2,Radial Velocity,1,763.000,2.60,19.84,2011
3,Radial Velocity,1,326.030,19.40,110.62,2007
4,Radial Velocity,1,516.220,10.50,119.47,2009


In [33]:
X = df.drop(labels=['method'],axis=1)
Y = df[['method']]

In [34]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1035 entries, 0 to 1034
Data columns (total 6 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   method          1035 non-null   object 
 1   number          1035 non-null   int64  
 2   orbital_period  992 non-null    float64
 3   mass            513 non-null    float64
 4   distance        808 non-null    float64
 5   year            1035 non-null   int64  
dtypes: float64(3), int64(2), object(1)
memory usage: 48.6+ KB


In [35]:
categorical_cols = X.select_dtypes(include='object').columns
numerical_cols = X.select_dtypes(exclude='object').columns

In [67]:
df['method'].unique()

array(['Radial Velocity', 'Imaging', 'Eclipse Timing Variations',
       'Transit', 'Astrometry', 'Transit Timing Variations',
       'Orbital Brightness Modulation', 'Microlensing', 'Pulsar Timing',
       'Pulsation Timing Variations'], dtype=object)

In [70]:
encoder = LabelEncoder()
df['method']= encoder.fit_transform(df['method'])

In [71]:
## Numerical Pipeline
num_pipeline=Pipeline(
    steps=[
    ('imputer',SimpleImputer(strategy='median')),
    ('scaler',StandardScaler())

    ]

)



preprocessor=ColumnTransformer([
('num_pipeline',num_pipeline,numerical_cols)
])

In [72]:
## Train test split

from sklearn.model_selection import train_test_split

X_train,X_test,y_train,y_test=train_test_split(X,Y,test_size=0.30,random_state=30)

In [73]:
X_train=pd.DataFrame(preprocessor.fit_transform(X_train),columns=preprocessor.get_feature_names_out())
X_test=pd.DataFrame(preprocessor.transform(X_test),columns=preprocessor.get_feature_names_out())

In [74]:
X_train.head()

,num_pipeline__sepal_length,num_pipeline__sepal_width,num_pipeline__petal_length,num_pipeline__petal_width
0,-0.750237,-0.900050,0.128061,0.311606
1,-0.256351,-0.900050,0.299717,0.178604
2,0.361008,-0.672326,0.585811,0.045601
3,-0.379822,-1.355496,0.185280,0.178604
4,-0.256351,-0.216879,0.242498,0.178604


In [75]:
from sklearn.ensemble import RandomForestClassifier
rfc=RandomForestClassifier(n_estimators=100,max_depth=10,random_state=42)
rfc.fit(X_train,y_train)

RandomForestClassifier(max_depth=10, random_state=42)

In [76]:
from sklearn.metrics import accuracy_score,precision_score,recall_score,f1_score
y_pred=rfc.predict(X_test)

In [77]:
print(accuracy_score(y_test,y_pred))

0.9333333333333333


In [78]:
import numpy as np
def evaluate_model(true, predicted):
    mae = mean_absolute_error(true, predicted)
    mse = mean_squared_error(true, predicted)
    rmse = np.sqrt(mean_squared_error(true, predicted))
    r2_square = r2_score(true, predicted)
    return mae, rmse, r2_square

In [79]:
## Train multiple models
## Model Ecaluation
from sklearn.linear_model import LogisticRegression
models={
    'rfc':RandomForestClassifier(random_state=42),
    'lr' : LogisticRegression(random_state=42)
}
trained_model_list=[]
model_list=[]
r2_list=[]

for i in range(len(list(models))):
    model=list(models.values())[i]
    model.fit(X_train,y_train)

    #Make Predictions
    y_pred=model.predict(X_test)

    accuracy_score(y_test,y_pred)

    print(list(models.keys())[i])
    model_list.append(list(models.keys())[i])

    print('Model Training Performance')
    print(accuracy_score(y_test,y_pred))

    
    print('='*35)
    print('\n')


rfc
Model Training Performance
0.9333333333333333


lr
Model Training Performance
0.9555555555555556




In [92]:
df2=load_dataset('iris')

In [93]:
df2.head()

,sepal_length,sepal_width,petal_length,petal_width,species
0,5.1,3.5,1.4,0.2,setosa
1,4.9,3.0,1.4,0.2,setosa
2,4.7,3.2,1.3,0.2,setosa
3,4.6,3.1,1.5,0.2,setosa
4,5.0,3.6,1.4,0.2,setosa


In [94]:
X = df2.drop(labels=['species'],axis=1)
Y = df2[['species']]

In [95]:
df2.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 150 entries, 0 to 149
Data columns (total 5 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   sepal_length  150 non-null    float64
 1   sepal_width   150 non-null    float64
 2   petal_length  150 non-null    float64
 3   petal_width   150 non-null    float64
 4   species       150 non-null    object 
dtypes: float64(4), object(1)
memory usage: 6.0+ KB


In [96]:
# Segregating numerical and categorical variables
categorical_cols = X.select_dtypes(include='object').columns
numerical_cols = X.select_dtypes(exclude='object').columns

In [97]:
encoder = LabelEncoder()
df2['species'] = encoder.fit_transform(df2['species'] )

In [98]:
## Numerical Pipeline
num_pipeline=Pipeline(
    steps=[
    ('imputer',SimpleImputer(strategy='median')),
    ('scaler',StandardScaler())

    ]

)



preprocessor=ColumnTransformer([
('num_pipeline',num_pipeline,numerical_cols)
])

In [99]:
Logistic = make_pipeline(preprocessor,LogisticRegression(random_state=1))
Forest = make_pipeline(preprocessor,RandomForestClassifier(random_state=1))

In [101]:
voting_classifier = VotingClassifier(
    estimators=[('LogisticRegresson', Logistic), ('RandomForest', Forest)],
    voting='hard')

In [102]:
from sklearn.model_selection import train_test_split

X_train,X_test,y_train,y_test=train_test_split(X,Y,test_size=0.30,random_state=30)

In [103]:
voting_classifier.fit(X_train,y_train)

VotingClassifier(estimators=[('LogisticRegresson',
                              Pipeline(steps=[('columntransformer',
                                               ColumnTransformer(transformers=[('num_pipeline',
                                                                                Pipeline(steps=[('imputer',
                                                                                                 SimpleImputer(strategy='median')),
                                                                                                ('scaler',
                                                                                                 StandardScaler())]),
                                                                                Index(['sepal_length', 'sepal_width', 'petal_length', 'petal_width'], dtype='object'))])),
                                              ('logisticregression',
                                               LogisticRegressi...),
                             ('RandomForest',
                              Pipeline(steps=[('columntransformer',
                                               ColumnTransformer(transformers=[('num_pipeline',
                                                                                Pipeline(steps=[('imputer',
                                                                                                 SimpleImputer(strategy='median')),
                                                                                                ('scaler',
                                                                                                 StandardScaler())]),
                                                                                Index(['sepal_length', 'sepal_width', 'petal_length', 'petal_width'], dtype='object'))])),
                                              ('randomforestclassifier',
                                               RandomForestClassifier(random_state=1))]))])

In [104]:
y_pred = voting_classifier.predict(X_test)

In [105]:
print(accuracy_score(y_test,y_pred))
print(confusion_matrix(y_test,y_pred))
print(classification_report(y_test,y_pred))

0.9555555555555556
[[13  0  0]
 [ 0 12  1]
 [ 0  1 18]]
              precision    recall  f1-score   support

      setosa       1.00      1.00      1.00        13
  versicolor       0.92      0.92      0.92        13
   virginica       0.95      0.95      0.95        19

    accuracy                           0.96        45
   macro avg       0.96      0.96      0.96        45
weighted avg       0.96      0.96      0.96        45

